In [1]:
import math
from collections import Counter

data = [
    {"Tageszeit": "Morgens", "Aktivität": "Arbeiten", "Freundin dabei": "Nein", "Wetter": "Sonnig", "KI-Lernen": "Nein"},
    {"Tageszeit": "Morgens", "Aktivität": "Arbeiten", "Freundin dabei": "Ja", "Wetter": "Regen", "KI-Lernen": "Ja"},
    {"Tageszeit": "Mittags", "Aktivität": "Sport", "Freundin dabei": "Ja", "Wetter": "Sonnig", "KI-Lernen": "Nein"},
    {"Tageszeit": "Mittags", "Aktivität": "Sport", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Nein"},
    {"Tageszeit": "Abends", "Aktivität": "Programmieren", "Freundin dabei": "Ja", "Wetter": "Sonnig", "KI-Lernen": "Ja"},
    {"Tageszeit": "Abends", "Aktivität": "Programmieren", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Ja"},
    {"Tageszeit": "Nachmittags", "Aktivität": "Lesen", "Freundin dabei": "Ja", "Wetter": "Sonnig", "KI-Lernen": "Ja"},
    {"Tageszeit": "Nachmittags", "Aktivität": "Lesen", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Ja"},
    {"Tageszeit": "Morgens", "Aktivität": "Lesen", "Freundin dabei": "Nein", "Wetter": "Sonnig", "KI-Lernen": "Nein"},
    {"Tageszeit": "Mittags", "Aktivität": "Arbeiten", "Freundin dabei": "Ja", "Wetter": "Regen", "KI-Lernen": "Nein"},
    {"Tageszeit": "Abends", "Aktivität": "Sport", "Freundin dabei": "Ja", "Wetter": "Sonnig", "KI-Lernen": "Nein"},
    {"Tageszeit": "Nachmittags", "Aktivität": "Arbeiten", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Ja"},
    {"Tageszeit": "Morgens", "Aktivität": "Arbeiten", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Ja"}
]

def gini_impurity(data, target_attr):
    """Berechnet die Gini-Impurity einer Liste basierend auf dem Zielattribut."""
    counts = Counter(row[target_attr] for row in data)
    total = len(data)
    return 1 - sum((count / total) ** 2 for count in counts.values())

def gini_gain(data, split_attr, target_attr):
    """Berechnet den Gini-Gewinn für ein gegebenes Attribut."""
    total_impurity = gini_impurity(data, target_attr)
    values = set(row[split_attr] for row in data)
    subset_impurity = 0

    for value in values:
        subset = [row for row in data if row[split_attr] == value]
        weight = len(subset) / len(data)
        subset_impurity += weight * gini_impurity(subset, target_attr)

    return total_impurity - subset_impurity

def id3(data, attributes, target_attr):
    """Rekursive Implementierung des ID3-Algorithmus basierend auf Gini-Gewinn."""
    # Abbruchbedingung: Wenn alle Zielwerte gleich sind
    target_values = [row[target_attr] for row in data]
    if len(set(target_values)) == 1:
        return target_values[0]

    # Abbruchbedingung: Wenn keine Attribute mehr übrig sind
    if not attributes:
        return Counter(target_values).most_common(1)[0][0]

    # Wähle das Attribut mit dem höchsten Gini-Gewinn
    gains = {attr: gini_gain(data, attr, target_attr) for attr in attributes}
    best_attr = max(gains, key=gains.get)

    tree = {best_attr: {}}
    values = set(row[best_attr] for row in data)

    for value in values:
        subset = [row for row in data if row[best_attr] == value]
        subtree = id3(subset, [attr for attr in attributes if attr != best_attr], target_attr)
        tree[best_attr][value] = subtree

    return tree

def print_tree(tree, indent=""):
    """Formatiert die Ausgabe des Entscheidungsbaums."""
    if isinstance(tree, dict):
        for attr, branches in tree.items():
            for value, subtree in branches.items():
                print(f"{indent}{attr} = {value}:")
                print_tree(subtree, indent + "  ")
    else:
        print(f"{indent}--> {tree}")

def evaluate(tree, sample):
    """Evaluieren eines Datensatzes mit dem Entscheidungsbaum."""
    if not isinstance(tree, dict):
        return tree

    attr = next(iter(tree))
    value = sample.get(attr)
    if value not in tree[attr]:
        return None  # Unbekannter Wert
    return evaluate(tree[attr][value], sample)


# ID3-Algorithmus anwenden
target_attr = "KI-Lernen"
attributes = [attr for attr in data[0].keys() if attr != target_attr]
decision_tree = id3(data, attributes, target_attr)

# Entscheidungsbaum ausgeben
print("Entscheidungsbaum:")
print_tree(decision_tree)

# Evaluierung eines Beispieldatensatzes
sample = {"Tageszeit": "Abends", "Aktivität": "Programmieren", "Freundin dabei": "Ja", "Essen": "Pizza"}
result = evaluate(decision_tree, sample)
print("\nEvaluierung des Beispiels:")
print(f"Beispiel: {sample}")
print(f"Ergebnis: {result}")


Entscheidungsbaum:
Tageszeit = Mittags:
  --> Nein
Tageszeit = Nachmittags:
  --> Ja
Tageszeit = Abends:
  Aktivität = Programmieren:
    --> Ja
  Aktivität = Sport:
    --> Nein
Tageszeit = Morgens:
  Wetter = Sonnig:
    --> Nein
  Wetter = Regen:
    --> Ja

Evaluierung des Beispiels:
Beispiel: {'Tageszeit': 'Abends', 'Aktivität': 'Programmieren', 'Freundin dabei': 'Ja', 'Essen': 'Pizza'}
Ergebnis: Ja
